In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1691454429219_0005,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import boto3
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('trusted_orders').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_ = '20230804'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
bucket_refined = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/004_refined'.format(dia=_DIA_)
bucket_trusted = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/003_trusted'.format(dia=_DIA_)
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
output_refined = 't_indicadores_tempo_medio_atraso_pagamento'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
path_lake = os.path.join('s3://',bucket_trusted,'trusted_bureau_20230805')
bureau_00=sqlContext.read.format("parquet").load(path_lake)
bureau_00.registerTempTable("bureau_00")
bureau_00.cache()
bureau_00.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1716428

In [9]:
bureau_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ts_proc: long (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: integer (nullable = true)
 |-- CREDIT_DAY_OVERDUE: integer (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: double (nullable = true)
 |-- DAYS_ENDDATE_FACT: double (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: double (nullable = true)
 |-- CNT_CREDIT_PROLONG: integer (nullable = true)
 |-- AMT_CREDIT_SUM: double (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: double (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: double (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: double (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: integer (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_proc_partition: long (nullable = true)

In [16]:
indicadores_00 = spark.sql(     
    """
        SELECT 
            SK_ID_CURR,
            CREDIT_DAY_OVERDUE,
            CASE
                WHEN CREDIT_DAY_OVERDUE BETWEEN 0 AND 90 THEN '0-90'
                WHEN CREDIT_DAY_OVERDUE BETWEEN 91 AND 180 THEN '90-180'
                WHEN CREDIT_DAY_OVERDUE BETWEEN 181 AND 360 THEN '180-360'
                WHEN CREDIT_DAY_OVERDUE BETWEEN 361 AND 720 THEN '360-720'
                ELSE '+720'
                END AS BUCKET_ATRASO
        FROM 
            bureau_00
        WHERE 
            CREDIT_DAY_OVERDUE > 0
        

    """)
indicadores_00.registerTempTable("indicadores_00")
indicadores_00.cache()
indicadores_00.count() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4217

In [17]:
indicadores_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+-------------+
|SK_ID_CURR|CREDIT_DAY_OVERDUE|BUCKET_ATRASO|
+----------+------------------+-------------+
|    282297|              2603|         +720|
|    408647|                 6|         0-90|
|    380677|                30|         0-90|
|    218135|              2156|         +720|
|    229323|               496|      360-720|
|    295460|               186|      180-360|
|    295460|              2264|         +720|
|    129289|                41|         0-90|
|    446640|                 8|         0-90|
|    446640|                 8|         0-90|
|    446640|                71|         0-90|
|    446640|                13|         0-90|
|    398626|               252|      180-360|
|    178283|              1201|         +720|
|    417930|                30|         0-90|
|    241004|                30|         0-90|
|    241004|                12|         0-90|
|    241004|                38|         0-90|
|    345385|              1777|   

In [18]:
tabela = output_refined

path_athena = os.path.join('s3://',bucket_refined,tabela)
print('path_tabela:', path_athena)
indicadores_00.coalesce(1).write.\
format("parquet").mode("overwrite").option("compression", "gzip").option("overwriteSchema", "true").\
option("path", path_athena).saveAsTable(tabela)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path_tabela: s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/004_refined/t_indicadores_tempo_medio_atraso_pagamento

### cria tabela athena

In [13]:
# Criando External Table
import boto3
client = boto3.client('athena',region_name='us-east-1')
config = {
    'OutputLocation': 's3://bkt-datalake-grupo-01-hackathon-podacademy/athena/log/'
}

sql = """ DROP TABLE IF EXISTS `t_indicadores_tempo_medio_atraso_pagamento`"""

context = {'Database': 'pod_academy'}
RES = client.start_query_execution(QueryString = sql, 
                             QueryExecutionContext = context,
                             ResultConfiguration = config)
print(sql)
print(RES)

# Query Execution Parameters
sql = """

CREATE EXTERNAL TABLE `t_indicadores_tempo_medio_atraso_pagamento`(
  'SK_ID_CURR' bigint,
  'CREDIT_DAY_OVERDUE' bigint,
  'BUCKET_ATRASO' string
  )
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/004_refined/t_indicadores_tempo_medio_atraso_pagamento'
TBLPROPERTIES (
  'has_encrypted_data'='true', 
  'transient_lastDdlTime'='1684958889')   
""".format(dia=_DIA_)

context = {'Database': 'pod_academy'}
client.start_query_execution(QueryString = sql, 
                             QueryExecutionContext = context,
                             ResultConfiguration = config)
print(sql)
print(RES)

The code failed because of a fatal error:
	Session 4 did not start up in 180 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.
